# 1. Quickstart

Run the following to check if the container is running:

```$ docker ps --format "table {{.Image}}\t{{.Names}}\t{{.Status}}" | grep gai-ttt```

Once it is started, the status will look like this:

```gai-ttt:latest   gai-ttt   Up 2 minutes```

Otherwise, run `gai docker start`.



a) Patch OpenAI Client

In [3]:
from rich.console import Console
console = Console()

import os
# Replace this with your OpenAI API key if you want to use it
if os.environ.get("OPENAI_API_KEY") is None:
    os.environ["OPENAI_API_KEY"] = ""

# Load and patch openai
from openai import OpenAI
openai_client = OpenAI()

from gai.openai.patch import patch_chatcompletions
openai_client = patch_chatcompletions(openai_client)

import json


b) Create Completions

In [2]:
# Use openai
if os.environ.get("OPENAI_API_KEY"):
    console.print("[yellow]Use GPT-4o create:[/]")
    response=openai_client.chat.completions.create(model="gpt-4o",messages=[{"role":"user","content":"Tell me a one sentence story"}])
    console.print(f"[white]{response}[/]")
    console.print("[yellow]extract:[/]")
    console.print(f"[yellow]{response.extract()}[/]")

# Use LlamaCPP
console.print("[bright_yellow]Use LlamaCPP:[/]")
response=openai_client.chat.completions.create(model="ttt-llamacpp-dolphin",messages=[{"role":"user","content":"Tell me a one sentence story"}])
console.print(f"[bright_white italic]{response}[/]")
console.print("[bright_yellow italic]extract:[/]")
console.print(f"[bright_yellow italic]{response.extract()}[/]")

Use GPT-4o create:

ChatCompletion(id='chatcmpl-Aplaaaq5499SoGwMrO7p6g1zF7neo', choices=[Choice(finish_reason='stop', index=0, 
logprobs=None, message=ChatCompletionMessage(content='In a world where dreams were traded like currency, young 
Elara discovered that her most cherished dream had the power to reshape reality.', refusal=None, role='assistant', 
function_call=None, tool_calls=None))], created=1736901024, model='gpt-4o-2024-08-06', object='chat.completion', 
service_tier='default', system_fingerprint='fp_4691090a87', usage=CompletionUsage(completion_tokens=27, 
prompt_tokens=13, total_tokens=40, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, 
completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 
'rejected_prediction_tokens': 0}), extract=<function attach_extractor.<locals>.<lambda> at 0x7fdea19c3a30>)

extract:

{'type': 'content', 'content': 'In a world where dreams were traded like currency, young Elara discovered that her 
most cherished dream had the power to reshape reality.'}

Use LlamaCPP:

ChatCompletion(id='chatcmpl-86ca481e-117d-4191-97b8-17dde31369b6', choices=[Choice(finish_reason='stop', index=0, 
logprobs=None, message=ChatCompletionMessage(content='The old man found a rare diamond in his backyard, changing 
his life forever.', refusal=None, role='assistant', function_call=None, tool_calls=None, audio=None))], 
created=1736901030, model='llamacpp', object='chat.completion', service_tier=None, system_fingerprint=None, 
usage=CompletionUsage(completion_tokens=17, prompt_tokens=21, total_tokens=38, completion_tokens_details=None, 
prompt_tokens_details=None), extract=<function attach_extractor.<locals>.<lambda> at 0x7fdea1f36e60>)

extract:

{'type': 'content', 'content': 'The old man found a rare diamond in his backyard, changing his life forever.'}

c) Stream Completions

In [3]:
# Use openai
if os.environ.get("OPENAI_API_KEY"):
    console.print("[yellow]Use GPT-4o:[/]")
    response=openai_client.chat.completions.create(model="gpt-4o",messages=[{"role":"user","content":"Tell me a one sentence story"}],stream=True)
    for chunk in response:
        chunk = chunk.extract()
        if chunk and type(chunk) is str:
            print(chunk,end="",flush=True)
print()
# Use exllama
console.print("[bright_yellow]Use LlamaCPP:[/]")
response=openai_client.chat.completions.create(model="ttt-llamacpp-dolphin",messages=[{"role":"user","content":"Tell me a one sentence story"}],stream=True)
for chunk in response:
    chunk = chunk.extract()
    if chunk and type(chunk) is str:
            print(chunk,end="",flush=True)



Use GPT-4o:

Beneath the whispering pines, a forgotten diary revealed secrets that forever changed the destiny of the small mountain town.


Use LlamaCPP:

An old man found a lost puppy and decided to adopt it, bringing joy and companionship to both their lives.

d) Create Tool Call

In [4]:
# Use openai
import json
if os.environ.get("OPENAI_API_KEY"):
    console.print("[yellow]Use GPT-4o:[/]")
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "google",
                    "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "search_query": {
                                "type": "string",
                                "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                            }
                        },
                        "required": ["search_query"]
                    }
                }
            }
        ],
        tool_choice="required"
    )
    console.print(f"[yellow italic]{json.dumps(response.extract(),indent=2)}[/]")
    print()

# Use exllama
console.print("[bright_yellow]Use LlamaCPP:[/]")
response = openai_client.chat.completions.create(
    model="ttt-llamacpp-dolphin",
    messages=[{"role": "user", "content": "What is the current time in Singapore?"},
              {"role": "assistant","content":""}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }
    ],
    tool_choice="required",
)
import json
console.print(f"[bright_yellow italic]{json.dumps(response.extract(),indent=2)}[/]")


Use GPT-4o:

{
  "type": "function",
  "name": "google",
  "arguments": "{\"search_query\":\"current time in Singapore\"}"
}

Use LlamaCPP:

{
  "type": "function",
  "name": "google",
  "arguments": "{\"search_query\": \"current time in Singapore\"}"
}

e) Structured Output

In [4]:
import json
from pydantic import BaseModel
class Book(BaseModel):
    title: str
    summary: str
    author: str
    published_year: int
response_model = Book.schema()

data = """Foundation is a science fiction novel by American writer
        Isaac Asimov. It is the first published in his Foundation Trilogy (later
        expanded into the Foundation series). Foundation is a cycle of five
        interrelated short stories, first published as a single book by Gnome Press
        in 1951. Collectively they tell the early story of the Foundation,
        an institute founded by psychohistorian Hari Seldon to preserve the best
        of galactic civilization after the collapse of the Galactic Empire."""

# Use openai
if os.environ.get("OPENAI_API_KEY"):

    console.print("[yellow]Use GPT-4o:[/]")

    response = openai_client.beta.chat.completions.parse(
        model="gpt-4o",
        response_format=Book,
        messages=[{"role": "user", "content": data}]
    )
    console.print(f"[yellow italic]{json.dumps(response.extract(), indent=2)}[/]")
    print()

# Use exllama
console.print("[bright_yellow]Use LlamaCPP:[/]")
response = openai_client.beta.chat.completions.parse(
    model="gpt-4o",
    response_format=Book,
    messages=[{"role": "user", "content": data}],
    timeout=60
)
import json
console.print(f"[yellow italic]{json.dumps(response.extract(), indent=2)}[/]")


# response = openai_client.chat.completions.create(
#     model="ttt-llamacpp-dolphin",
#     response_format=Book,
#     messages=[{"role": "user", "content": data}],
#     timeout=60
# )
# import json
# console.print(f"[bright_yellow italic]{json.dumps(response.extract(), indent=2)}[/]")


Use GPT-4o:

{
  "type": "content",
  "content": "{\"title\":\"Foundation\",\"summary\":\"Set in a decaying Galactic Empire, \\\"Foundation\\\" follows
the visionary psychohistorian Hari Seldon who foresees its inevitable fall. He devises a scientific discipline 
known as \\\"psychohistory\\\" to predict social trends and statistically forecast the future outcomes of large 
populations. Seldon creates the Foundation, a group of exceptional individuals tasked with preserving the knowledge
of civilization in order to shorten the impending dark age following the empire's collapse.  \\n\\nAs the empire's 
influence wanes and chaos ensues, central themes of knowledge, power, and adaptability permeate the narrative. The 
book consists of five interconnected short stories, exploring how the Foundation evolves over generations in 
response to challenges, threats, and their mission to create a better future. Each segment showcases the ingenuity,
resilience, and conflicts faced by the guardians of civilization, as they strive to maintain order amidst turmoil. 
\\n\\nThrough these stories, Asimov delves into the intricacies of human behavior, politics, and the concept of 
predicting historical outcomes, all wrapped seamlessly in a riveting and meticulously constructed universe. As the 
story unfolds, the reader is invited to reflect on the interplay between fate, free will, and the role of science 
and innovation in shaping destiny.\",\"author\":\"Isaac Asimov\",\"published_year\":1951}"
}

Use LlamaCPP:

{
  "type": "content",
  "content": "{\"title\":\"Foundation\",\"summary\":\"Set in a future Galactic Empire, the novel introduces readers
to Hari Seldon, a psychohistorian who has developed a scientific method to predict the future by analyzing mass 
human behavior. Seldon foresees the imminent fall of the Empire, which will lead to a dark age lasting thousands of
years. To shorten this period of turmoil, Seldon establishes the Foundation, a collective of the Empire's greatest 
minds tasked with preserving and advancing knowledge and culture. The story unfolds in five interconnected 
segments, documenting pivotal events that lead to the establishment, survival, and expansion of the Foundation. 
Each section presents challenges faced by the Foundation, showcasing political intrigue, scientific innovation, and
the battle between free will and deterministic fate.\",\"author\":\"Isaac Asimov\",\"published_year\":1951}"
}